In [1]:
! jupyter kernelspec list

Available kernels:
  logistic    /Users/chazarey/Library/Jupyter/kernels/logistic
  python3     /Users/chazarey/Desktop/C/DL/DeepLearning/pyenv/share/jupyter/kernels/python3
